# 필요한 라이브러리 import

In [2]:
import csv
import pandas as pd
import os
import re
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from collections import defaultdict
import warnings
plt.rc('font',family='Malgun Gothic')
plt.rc('font',size=10)
from sklearn.metrics import mean_absolute_error
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

from tensorflow.keras.models import load_model
import tensorflow as tf

warnings.filterwarnings(action='ignore')

In [3]:
with open('./models/공격수_ss.pickle','rb') as f:
    forward_ss = pickle.load(f)
    
with open('./models/수비수_ss.pickle','rb') as f:
    defender_ss = pickle.load(f)
    
with open('./models/미드필더_ss.pickle','rb') as f:
    midfielder_ss = pickle.load(f)
    
with open('./models/공격수_pca.pickle','rb') as f:
    forward_pca = pickle.load(f)
    
with open('./models/수비수_pca.pickle','rb') as f:
    defender_pca = pickle.load(f)    

with open('./models/미드필더_pca.pickle','rb') as f:
    midfielder_pca = pickle.load(f)
    
forward_model = load_model('./models/공격수_nn.h5')
midfielder_model = load_model('./models/미드필더_nn.h5')
defender_model = load_model('./models/수비수_nn.h5')
    
premier_df = pd.read_csv('./datas/NEW_FM_DATA_MERGED/2021-2022/2021-2022-PremierLeague.csv')
premier_df.head(1)

,이름,시즌,포지션,구단,리그,골,도움,임대 기간,승,무,...,공중 A/90,태클 성공률,경기당 패스 시도/90,경기당 패스 성공/90,K Ps/90,태클 성공/90,드리블 성공/90,헤더 성공/90,Cr A/90,평균획득승점/경기
0,Teemu Pukki,2021-2022,공격수,노리치,잉글랜드 프리미어 리그,7,2,0,7,11,...,7.28,100,30.64,26.0,0.8,0.88,0.98,3.39,0.58,0.94


In [4]:
premier_df.rename(columns={'Cr C/A':'크로스 성공률','FA':'파울 획득','Int/90':'인터셉트/90','공중 A/90':'공중볼시도/90','K Ps/90':'키패스/90','Cr A/90':'크로스 시도/90'},inplace=True)
premier_df = premier_df.astype({'골':np.int64,'도움':np.int64,'파울 획득':np.int64,'태클 성공률':np.int64})
premier_df.head(1)

,이름,시즌,포지션,구단,리그,골,도움,임대 기간,승,무,...,공중볼시도/90,태클 성공률,경기당 패스 시도/90,경기당 패스 성공/90,키패스/90,태클 성공/90,드리블 성공/90,헤더 성공/90,크로스 시도/90,평균획득승점/경기
0,Teemu Pukki,2021-2022,공격수,노리치,잉글랜드 프리미어 리그,7,2,0,7,11,...,7.28,100,30.64,26.0,0.8,0.88,0.98,3.39,0.58,0.94


In [5]:
forward_df = premier_df[premier_df['포지션']=='공격수']
midfielder_df = premier_df[premier_df['포지션']=='미드필더']
defender_df = premier_df[premier_df['포지션']=='수비수']

In [6]:
use_col = ['골','도움','크로스 성공률','파울 획득','반칙 수/90', '기회 창출/90', '경기당 유효 슈팅/90', '경기당 슈팅/90', '인터셉트/90',
       '달린 거리/90분', '공중볼시도/90', '태클 성공률', '경기당 패스 시도/90', '경기당 패스 성공/90',
       '키패스/90', '태클 성공/90', '드리블 성공/90', '헤더 성공/90', '크로스 시도/90','평균획득승점/경기']

In [7]:
forward_df.loc[:,use_col[:-1]] = forward_ss.transform(forward_df.loc[:,use_col[:-1]])
midfielder_df.loc[:,use_col[:-1]] = midfielder_ss.transform(midfielder_df.loc[:,use_col[:-1]])
defender_df.loc[:,use_col[:-1]] = defender_ss.transform(defender_df.loc[:,use_col[:-1]])

In [8]:
forward_df.loc[:,use_col[:-1]] = forward_pca.transform(forward_df.loc[:,use_col[:-1]])
midfielder_df.loc[:,use_col[:-1]] = midfielder_pca.transform(midfielder_df.loc[:,use_col[:-1]])
defender_df.loc[:,use_col[:-1]] = defender_pca.transform(defender_df.loc[:,use_col[:-1]])

In [9]:
forward_df.loc[:,use_col[-1]]=forward_model.predict(tf.constant(forward_df.loc[:,use_col[:-1]].values))
midfielder_df.loc[:,use_col[-1]]=midfielder_model.predict(tf.constant(midfielder_df.loc[:,use_col[:-1]].values))
defender_df.loc[:,use_col[-1]]=defender_model.predict(tf.constant(defender_df.loc[:,use_col[:-1]].values))

In [10]:
forward_min = min(forward_df['평균획득승점/경기'].tolist())
midfielder_min = min(midfielder_df['평균획득승점/경기'].tolist())
defender_min = min(defender_df['평균획득승점/경기'].tolist())

In [11]:
forward_df['WAR'] = forward_df['평균획득승점/경기'].map(lambda x : x-forward_min+1)
midfielder_df['WAR'] = midfielder_df['평균획득승점/경기'].map(lambda x : x-midfielder_min+1)
defender_df['WAR'] = defender_df['평균획득승점/경기'].map(lambda x : x-defender_min+1)

In [12]:
all_df = pd.concat([forward_df,midfielder_df,defender_df])
all_df

,이름,시즌,포지션,구단,리그,골,도움,임대 기간,승,무,...,공중볼시도/90,태클 성공률,경기당 패스 시도/90,경기당 패스 성공/90,키패스/90,태클 성공/90,드리블 성공/90,헤더 성공/90,크로스 시도/90,평균획득승점/경기
0,Teemu Pukki,2021-2022,공격수,노리치,잉글랜드 프리미어 리그,1.029580,-0.191765,0,7,11,...,0.065793,-0.713096,0.085785,0.035830,0.116508,-0.010428,0.291732,0.301885,0.046450,1.169922
3,Milot Rashica,2021-2022,공격수,노리치,잉글랜드 프리미어 리그,-3.617324,-0.138959,0,8,9,...,-0.905892,-1.266459,-0.319230,0.968402,0.211179,-0.330103,0.339551,0.344472,0.068866,1.260483
4,Christos Tzolis,2021-2022,공격수,노리치,잉글랜드 프리미어 리그,-0.307845,-2.087460,0,7,7,...,-0.000531,0.448636,0.128777,0.160195,-0.314032,0.145894,0.088229,0.166498,0.120650,1.270277
11,Callum Wilson,2021-2022,공격수,뉴캐슬,잉글랜드 프리미어 리그,3.373397,0.346909,0,20,5,...,-0.246822,-0.300967,-0.277388,0.312659,0.759946,0.228488,-0.237908,-0.209841,0.167163,2.038201
14,Allan Saint-Maximin,2021-2022,공격수,뉴캐슬,잉글랜드 프리미어 리그,-2.523324,0.794901,0,22,6,...,0.657919,-1.310765,0.675606,-0.619859,-0.550243,-0.404145,0.024166,0.146093,0.037275,1.984892
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
291,Japhet Tanganga,2021-2022,수비수,토트넘,잉글랜드 프리미어 리그,3.453112,-1.628559,0,18,8,...,-0.158645,0.393515,0.221558,0.586290,0.090621,0.338196,0.016689,0.064101,0.154106,2.376685
292,Davinson Sánchez,2021-2022,수비수,토트넘,잉글랜드 프리미어 리그,-1.229028,1.474621,0,17,9,...,0.394238,0.171823,0.000074,-0.464654,0.242468,0.105898,-0.152872,0.254630,-0.019095,2.492892
294,Eric Dier,2021-2022,수비수,토트넘,잉글랜드 프리미어 리그,0.808505,-0.154644,0,12,5,...,0.302790,-0.515614,-0.386957,0.716689,-0.466496,0.573868,-0.159115,0.169845,-0.059747,2.355036
295,Ben Davies,2021-2022,수비수,토트넘,잉글랜드 프리미어 리그,7.224929,-5.205536,0,11,3,...,1.469321,-1.092427,-1.038637,-0.831584,1.318761,-0.031734,-0.499731,-0.538898,0.419938,2.600849


In [13]:
premier_df['WAR'] = all_df.sort_index()['평균획득승점/경기']

In [26]:
premier_df[premier_df['포지션']=='공격수'].sort_values(by='WAR',ascending=False).reset_index(drop=True).head(60)

,이름,시즌,포지션,구단,리그,골,도움,임대 기간,승,무,...,태클 성공률,경기당 패스 시도/90,경기당 패스 성공/90,키패스/90,태클 성공/90,드리블 성공/90,헤더 성공/90,크로스 시도/90,평균획득승점/경기,WAR
0,Phil Foden,2021-2022,공격수,맨시티,잉글랜드 프리미어 리그,8,5,0,21,2,...,77,42.61,37.41,1.79,2.45,2.21,3.09,4.57,2.50,2.725721
1,Riyad Mahrez,2021-2022,공격수,맨시티,잉글랜드 프리미어 리그,7,7,0,20,3,...,73,36.95,32.03,2.17,2.15,2.58,3.03,5.97,2.42,2.648051
2,Ferran Torres,2021-2022,공격수,맨시티,잉글랜드 프리미어 리그,7,2,0,14,3,...,73,33.36,28.50,1.13,1.21,1.34,6.46,1.27,2.05,2.280120
3,Kai Havertz,2021-2022,공격수,첼시,잉글랜드 프리미어 리그,22,8,0,34,11,...,83,40.82,35.29,1.88,1.54,1.65,4.63,2.11,2.05,2.270306
4,Romelu Lukaku,2021-2022,공격수,첼시,잉글랜드 프리미어 리그,36,0,0,36,10,...,71,30.22,25.83,0.80,0.52,1.15,7.00,0.78,2.03,2.263019
5,Christian Pulisic,2021-2022,공격수,첼시,잉글랜드 프리미어 리그,5,5,0,26,6,...,93,35.27,30.43,1.90,2.80,3.30,4.37,6.49,2.00,2.229352
6,Anthony Martial,2021-2022,공격수,첼시,잉글랜드 프리미어 리그,6,0,0,19,7,...,74,32.55,26.84,1.16,4.22,2.13,11.72,2.33,2.00,2.224028
7,Mohamed Salah,2021-2022,공격수,리버풀,잉글랜드 프리미어 리그,19,8,0,32,13,...,84,35.51,29.94,1.84,1.47,2.21,4.02,6.28,1.98,2.207082
8,Roberto Firmino,2021-2022,공격수,리버풀,잉글랜드 프리미어 리그,14,6,0,26,9,...,74,29.58,25.23,1.38,0.85,0.95,5.32,0.73,1.93,2.160165
9,Sadio Mané,2021-2022,공격수,리버풀,잉글랜드 프리미어 리그,13,4,0,25,11,...,82,38.51,34.24,1.39,2.77,2.88,6.04,5.14,1.91,2.137672


In [15]:
premier_df.query(f'이름 in ["Phil Foden"]')[['이름','WAR']]

,이름,WAR
80,Phil Foden,2.725721


In [21]:
my_players = ["Thiago Silva",
              "Kieran Trippier",
              "Antonio Rüdiger",
              "Kostas Tsimikas",
              "Marcos Alonso",
              "Oliver Skipp",              
              "Federico Bernardeschi",
              "Kieran Trippier",
              "Edinson Cavani",
              "Callum Wilson",
              "Andreas Christensen",
              "Seamus Coleman",
              "Issa Diop",
              "Abdoulaye Doucouré",
              "Demarai Gray",
              "Isaac Hayden",
              "Curtis Jones",
              "Noussair Mazraoui",
              "James Milner",
              "Paulinho",
              "Matt Ritchie",
              "Jonjo Shelvey"]
my_df = premier_df.query(f'이름 in {my_players}')[['이름','WAR']]

display(my_df.sort_values(by="WAR",ascending=False).head(10))
print("WAR 평균",np.mean(my_df["WAR"]))

,이름,WAR
62,Kostas Tsimikas,2.767686
13,Matt Ritchie,2.657593
264,Andreas Christensen,2.623056
169,Federico Bernardeschi,2.539080
260,Marcos Alonso,2.514795
262,Antonio Rüdiger,2.497346
267,Thiago Silva,2.471005
23,Jonjo Shelvey,2.468247
53,Noussair Mazraoui,2.393328
56,James Milner,2.361265


WAR 평균 2.3159977992375693


In [72]:
cpfc_players = ["Nathaniel Clyne","Odsonne Edouard","Ebere Eze","Marc Guehi","Will Hughes",
               "Cheikhou Kouyaté","Luka Milivojević","Tyrick Mitchell","Michael Olise","Jairo Riedewald",
               "Wilfried Zaha","Joachim Andersen"]


cpfc_df = premier_df.query(f'이름 in {cpfc_players}')[['이름','WAR']].sort_values(by="WAR",ascending=False)

display(cpfc_df.sort_values(by="WAR",ascending=False).head(10))
print("WAR 평균",np.mean(cpfc_df["WAR"]))

,이름,WAR
73,Cheikhou Kouyaté,2.146257
167,Tyrick Mitchell,1.782043
192,Michael Olise,1.704212
201,Ebere Eze,1.647425
203,Marc Guehi,1.638107
219,Wilfried Zaha,1.561346
221,Odsonne Edouard,1.547845
224,Luka Milivojević,1.543938
229,Joachim Andersen,1.513269
231,Nathaniel Clyne,1.508818


WAR 평균 1.622769563322563
